# Train binary classifier
Notebook to train and save a binary classifier that it will be used to identify energy consumption patents.

In [0]:
FEATURES_STORAGE_NAME = "challengebasf"
FEATURES_CONTAINER_NAME = "featurizeddata"
FEATURES_OUTPUT_FOLDER = "output_data"

ENERGY_CLASSIFIER_VERSION = "v1"  # TODO it shoud be increased automatically checking previous models stored
ENERGY_CLASSIFIER_STORAGE_NAME = FEATURES_STORAGE_NAME
ENERGY_CLASSIFIER_CONTAINER_NAME = "classifier"
ENERGY_CLASSIFIER_OUTPUT_FOLDER = f"output_data_{ENERGY_CLASSIFIER_VERSION}"

ENERGY_PATENTS_STORAGE_NAME = FEATURES_STORAGE_NAME
ENERGY_PATENTS_CONTAINER_NAME = "energypatents"
ENERGY_PATENTS_OUTPUT_FOLDER = f"output_data_{ENERGY_CLASSIFIER_VERSION}"

In [0]:
from azure.storage.blob import BlobServiceClient
from azure.core.exceptions import ResourceExistsError
def create_if_not_exists_container(storage_name: str, container_name: str):
    input_url = f"https://{storage_name}.blob.core.windows.net/"
    key = spark.conf.get(f"spark.hadoop.fs.azure.account.key.{storage_name}.blob.core.windows.net")
    service = BlobServiceClient(account_url=input_url, credential=key)
    container = service.get_container_client(container_name)
    try:
        container.create_container()
        print(f"Creating container: {container_name}")
    except ResourceExistsError:
        print("Output container already exists")

In [0]:
SECTIONS_IPCR = ["A", "B", "C", "D", "E", "F", "G"]
SECTIONS_CLASS_IPCR = [
    "A01", "A21", "A22", "A23", "A24", "A41", "A42", "A43", "A44", "A45", "A46", "A47", "A61", "A62", "A63", "A99",
    "B01", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B09",
    "B21", "B22", "B23", "B24", "B25", "B26", "B27", "B28", "B29", "B30", "B31", "B32",
    "B41", "B42", "B43", "B44", "B60", "B61", "B62", "B63", "B64", "B65", "B66", "B67", "B68", "B81", "B82", "B99",
    "C01", "C02", "C03", "C04", "C05", "C06", "C07", "C08", "C09", "C10", "C11", "C12", "C13", "C14",
    "C21", "C22", "C23", "C25", "C30", "C40", "C99",
    "D01", "D02", "D03", "D04", "D05", "D06", "D07", "D21", "D99",
    "E01", "E02", "E03", "E04", "E05", "E06", "E21", "E99",
    "F01", "F02", "F03", "F04", "F15", "F16", "F17", "F21",
    "F22", "F23", "F24", "F25", "F26", "F27", "F28", "F41", "F42", "F99",
    "G01", "G02", "G03", "G04", "G05", "G06", "G07", "G08", "G09", "G10", "G11", "G12", "G21", "G99",
    "H01", "H02", "H03", "H04", "H05", "H99",
]

FEATURE_COLS_SECTION = [f"section_{section}" for section in SECTIONS_IPCR]
FEATURE_COLS_SECTION_CLASS = [f"section_class_{section_class}" for section_class in SECTIONS_CLASS_IPCR]

NON_TEXT_FEATURE_COLS = FEATURE_COLS_SECTION + FEATURE_COLS_SECTION_CLASS
COL_ENGLISH_TEXT = "english_text_features"
TEXT_FEATURE_COLS = ["flag_energy_title", "flag_energy_abstract", "flag_energy_claims"]

In [0]:
from pyspark.sql import functions as sf
input_container_path = f"wasbs://{FEATURES_CONTAINER_NAME}@{FEATURES_STORAGE_NAME}.blob.core.windows.net"
input_path = f"{input_container_path}/{FEATURES_OUTPUT_FOLDER}/"
input_data = spark.read.parquet(input_path)
input_data = input_data.select("_file", COL_ENGLISH_TEXT, *NON_TEXT_FEATURE_COLS, *TEXT_FEATURE_COLS)

input_container_path = f"wasbs://trainingsamples@challengebasf.blob.core.windows.net"
input_blob_folder = f"{input_container_path}/file_names/"
target_files = spark.read.parquet(input_blob_folder)
target_file_p = target_files.toPandas()
target_file_names = target_file_p["_file"].tolist()

input_data = input_data.filter(sf.col("_file").isin(target_file_names))
input_data = input_data.join(target_files, ["_file"], "left")

In [0]:
positive_data = input_data.filter(sf.col("positive_data") == 1)
negative_data = input_data.filter(sf.col("positive_data") == 0)
(p_training_data, p_test_data) = positive_data.randomSplit([0.8, 0.2], seed=18)
(n_training_data, n_test_data) = negative_data.randomSplit([0.8, 0.2], seed=18)
p_training_data = p_training_data.cache()
num_ps_training = p_training_data.count()
num_ns_training = n_training_data.count()
print(f"Positive registers for training: {num_ps_training}")
print(f"Negative registers for training: {num_ns_training}")
# First implementation to balance data, more advanced techniques as SMOTH should be analyzed
num_dups = num_ns_training // num_ps_training
print(f"Duplicating positive data {num_dups} times to balance the data")
p_training_data = p_training_data.withColumn("dummy", sf.explode(sf.array(*[sf.lit(n) for n in range(num_dups)])))
p_training_data = p_training_data.drop("dummy")

training_data = p_training_data.union(n_training_data)
test_data = p_test_data.union(n_test_data)

Positive registers for training: 46
Negative registers for training: 477
Duplicating positive data 10 times to balance the data

In [0]:
training_data.write.mode("overwrite").parquet("/tmp/training")
test_data.write.mode("overwrite").parquet("/tmp/test")

## Training the model
We start with a simple model, LogisticRegresson.
Other models that should be tried are NaiveBayes and RandomForest

In [0]:
training_data.printSchema()

root
-- _file: string (nullable = true)
-- section_A: integer (nullable = true)
-- section_B: integer (nullable = true)
-- section_C: integer (nullable = true)
-- section_D: integer (nullable = true)
-- section_E: integer (nullable = true)
-- section_F: integer (nullable = true)
-- section_G: integer (nullable = true)
-- section_class_A01: integer (nullable = true)
-- section_class_A21: integer (nullable = true)
-- section_class_A22: integer (nullable = true)
-- section_class_A23: integer (nullable = true)
-- section_class_A24: integer (nullable = true)
-- section_class_A41: integer (nullable = true)
-- section_class_A42: integer (nullable = true)
-- section_class_A43: integer (nullable = true)
-- section_class_A44: integer (nullable = true)
-- section_class_A45: integer (nullable = true)
-- section_class_A46: integer (nullable = true)
-- section_class_A47: integer (nullable = true)
-- section_class_A61: integer (nullable = true)
-- section_class_A62: integer (nullable = true)
-- section_class_A63: integer (nullable = true)
-- section_class_A99: integer (nullable = true)
-- section_class_B01: integer (nullable = true)
-- section_class_B02: integer (nullable = true)
-- section_class_B03: integer (nullable = true)
-- section_class_B04: integer (nullable = true)
-- section_class_B05: integer (nullable = true)
-- section_class_B06: integer (nullable = true)
-- section_class_B07: integer (nullable = true)
-- section_class_B08: integer (nullable = true)
-- section_class_B09: integer (nullable = true)
-- section_class_B21: integer (nullable = true)
-- section_class_B22: integer (nullable = true)
-- section_class_B23: integer (nullable = true)
-- section_class_B24: integer (nullable = true)
-- section_class_B25: integer (nullable = true)
-- section_class_B26: integer (nullable = true)
-- section_class_B27: integer (nullable = true)
-- section_class_B28: integer (nullable = true)
-- section_class_B29: integer (nullable = true)
-- section_class_B30: integer (nullable = true)
-- section_class_B31: integer (nullable = true)
-- section_class_B32: integer (nullable = true)
-- section_class_B41: integer (nullable = true)
-- section_class_B42: integer (nullable = true)
-- section_class_B43: integer (nullable = true)
-- section_class_B44: integer (nullable = true)
-- section_class_B60: integer (nullable = true)
-- section_class_B61: integer (nullable = true)
-- section_class_B62: integer (nullable = true)
-- section_class_B63: integer (nullable = true)
-- section_class_B64: integer (nullable = true)
-- section_class_B65: integer (nullable = true)
-- section_class_B66: integer (nullable = true)
-- section_class_B67: integer (nullable = true)
-- section_class_B68: integer (nullable = true)
-- section_class_B81: integer (nullable = true)
-- section_class_B82: integer (nullable = true)
-- section_class_B99: integer (nullable = true)
-- section_class_C01: integer (nullable = true)
-- section_class_C02: integer (nullable = true)
-- section_class_C03: integer (nullable = true)
-- section_class_C04: integer (nullable = true)
-- section_class_C05: integer (nullable = true)
-- section_class_C06: integer (nullable = true)
-- section_class_C07: integer (nullable = true)
-- section_class_C08: integer (nullable = true)
-- section_class_C09: integer (nullable = true)
-- section_class_C10: integer (nullable = true)
-- section_class_C11: integer (nullable = true)
-- section_class_C12: integer (nullable = true)
-- section_class_C13: integer (nullable = true)
-- section_class_C14: integer (nullable = true)
-- section_class_C21: integer (nullable = true)
-- section_class_C22: integer (nullable = true)
-- section_class_C23: integer (nullable = true)
-- section_class_C25: integer (nullable = true)
-- section_class_C30: integer (nullable = true)
-- section_class_C40: integer (nullable = true)
-- section_class_C99: integer (nullable = true)
-- section_class_D01: integer (nullable = true)
-- section_class_D02: integer (nullable = true)
-- section_class_D03: integer (n

In [0]:
training_data = spark.read.parquet("/tmp/training")
test_data = spark.read.parquet("/tmp/test")

In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler, CountVectorizer, IDF


minDF = 5
cv = CountVectorizer(inputCol=COL_ENGLISH_TEXT, outputCol="text_features_count", minDF=minDF)
idf = IDF(inputCol="text_features_count", outputCol="text_features_idf")
assembler = VectorAssembler(
    inputCols=NON_TEXT_FEATURE_COLS + TEXT_FEATURE_COLS + ["text_features_idf"],
    outputCol="features")
lr = LogisticRegression(elasticNetParam=1, regParam=0.05, labelCol="positive_data")

pipeline = Pipeline(stages=[cv, idf, assembler, lr])

model = pipeline.fit(training_data)

lrModel = model.stages[-1]

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

Coefficients: (17478,[133,136,166,208,235,256,395,414,443,453,454,752,794,831,889,1035,1430,1782,2278,2279,2452,2580,3726,3803,3893,4278,4725,6080,6911,7127,7216,8005,8180,8920,10498],[1.045317078209971,3.200630626025215,0.0003751432599678893,0.023757256362857308,-0.005816843579571505,0.0003194361795000194,0.06840872669764915,0.00048436778977061125,0.006427164624884581,0.0012929246329596979,0.003152220591618835,0.0021064689015935878,0.0031702881547655735,0.0085531468120577,0.0809517413949543,0.008747582508637167,0.006134710098839085,0.032577665220158175,0.005657371633980174,0.005336938116180936,0.03560337788455805,0.046665267614545314,0.06613672371088758,0.027152141741908886,0.12354005975253918,0.07246138224721625,0.037699978504925485,0.1687120961180183,0.016353963160640427,0.016353963160640427,0.17425519974466433,0.06612655382136758,0.0476333961074108,0.006342062537757665,0.19484002260780206])
Intercept: -1.7326075409623471

In [0]:
# Extract the summary from the returned LogisticRegressionModel instance trained in the earlier example
trainingSummary = lrModel.summary

# Obtain the objective per iteration
objectiveHistory = trainingSummary.objectiveHistory
print("objectiveHistory:")
for objective in objectiveHistory:
    print(objective)

# Obtain the receiver-operating characteristic as a dataframe and areaUnderROC.
trainingSummary.roc.show()
print("areaUnderROC: " + str(trainingSummary.areaUnderROC))

# Set the model threshold to maximize F-Measure
fMeasure = trainingSummary.fMeasureByThreshold
maxFMeasure = fMeasure.groupBy().max('F-Measure').select('max(F-Measure)').head()
bestThreshold = fMeasure.where(fMeasure['F-Measure'] == maxFMeasure['max(F-Measure)']) \
    .select('threshold').head()['threshold']
lr.setThreshold(bestThreshold)

objectiveHistory:
0.6929825871306489
0.6838016451896424
0.6704820636014295
0.6566649591583891
0.6371916870994745
0.6102295295369357
0.5876459098916551
0.5737385690178176
0.5636888992079162
0.5447445323037661
0.524583538692984
0.5090247730578928
0.5057608319759022
0.49804617072417057
0.48168514261121037
0.47685692253132894
0.47192482016574366
0.46708492214812763
0.4616624665680811
0.45582927713646326
0.4542233977805962
0.4535175711798571
0.4519685305686803
0.4512865533811438
0.45032854809023315
0.44928713022531463
0.44866278824799155
0.4484740642390786
0.4482568229676655
0.44800516915939514
0.44777022878239753
0.4474303549967854
0.4472107511940234
0.4469152659470239
0.4467605616546429
0.4466042483604137
0.4464460254829592
0.4463787046364266
0.44626656192801994
0.44619040566507073
0.44585436371710396
0.4456744031721833
0.44540894116339697
0.4450056765116484
0.44459260766820086
0.4444689391200636
0.44432313053178507
0.4441491433886109
0.44379177256058444
0.4434753245099012
0.4433496742434586
0.4432615465299544
0.4431093427605794
0.4431015678947086
0.4430058253609343
0.44300705410205715
0.44295562634451396
0.4429436713146775
0.44292519435234784
0.4429018718646287
0.4428771722453463
0.4428638403173677
0.4428432636166786
0.44281043279066484
0.4427389443834892
0.4426976060845432
0.4426759292300208
0.4426665455961018
0.44264465102674133
0.44262661234549916
0.44259209291923174
0.4425823178159812
0.44255167224141956
0.4425466949524333
0.44253940794800195
0.44252473300559586
0.4425158457636706
0.442489437343723
0.4424800071689567
0.44246086146450514
0.44244426904979317
0.44241046010484464
0.44240154285105543
0.44239577479595377
0.442383384957763
0.4423643485257162
0.4423402528591698
0.44232268229777794
0.4423187897842182
0.44231166292499596
0.442305484561519
0.44230239222964457
0.44229084601657875
0.4422821125429981
0.44227569891019064
0.4422719847651311
0.4422603470120441
0.4422535967936517
0.4422503572344191
0.4422438304222063
0.44223842667807156
+--------------------+-------------------+
 FPR| TPR|
+--------------------+-------------------+
 0.0| 0.0|
 0.0|0.10869565217391304|
 0.0|0.21739130434782608|
 0.0|0.30434782608695654|
 0.0| 0.391304347826087|
 0.0| 0.4782608695652174|
0.002096436058700...| 0.5434782608695652|
0.004192872117400...| 0.6086956521739131|
0.004192872117400...| 0.717391304347826|
0.004192872117400...| 0.8043478260869565|
0.006289308176100629| 0.8695652173913043|
0.006289308176100629| 0.9565217391304348|
0.010482180293501049| 1.0|
0.018867924528301886| 1.0|
0.027253668763102725| 1.0|
 0.03563941299790356| 1.0|
 0.0440251572327044| 1.0|
 0.05241090146750524| 1.0|
 0.06079664570230608| 1.0|
 0.06918238993710692| 1.0|
+--------------------+-------------------+
only showing top 20 rows

areaUnderROC: 0.9976529031081942
Out[34]: LogisticRegression_c31eef2b1ae4

## Test

In [0]:
test_data.printSchema

In [0]:
predictions = model.transform(test_data)
predicions.cache()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3665479319367413> in <module> 
 1 predictions = model . transform ( test_data ) 
 ----> 2 predicions . cache ( ) 

 NameError : name 'predicions' is not defined

In [0]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics, MulticlassMetrics
from pyspark.sql.types import DoubleType

predictionAndTarget = predictions.select(sf.col("positive_data").astype(DoubleType()), sf.col("prediction").astype(DoubleType()))

# Create both evaluators
metrics_binary = BinaryClassificationMetrics(predictionAndTarget.rdd.map(tuple))
metrics_multi = MulticlassMetrics(predictionAndTarget.rdd.map(tuple))

acc = metrics_multi.accuracy
f1 = metrics_multi.fMeasure(1.0)
precision = metrics_multi.precision(1.0)
recall = metrics_multi.recall(1.0)
auc = metrics_binary.areaUnderROC

In [0]:
print(f"Acc: {acc}")
print(f"f1: {f1}")
print(f"precision: {precision}")
print(f"Recall: {recall}")
print(f"Auc: {auc}")

Acc: 0.9708029197080292
f1: 0.8461538461538461
precision: 0.8461538461538461
Recall: 0.8461538461538461
Auc: 0.9150124069478909

### Save the model

In [0]:
create_if_not_exists_container(ENERGY_CLASSIFIER_STORAGE_NAME, ENERGY_CLASSIFIER_CONTAINER_NAME)
output_container_path = f"wasbs://{ENERGY_CLASSIFIER_CONTAINER_NAME}@{ENERGY_CLASSIFIER_STORAGE_NAME}.blob.core.windows.net"
output_blob_folder = f"{output_container_path}/{ENERGY_CLASSIFIER_OUTPUT_FOLDER}/"
model.save(output_blob_folder)

Output container already exists